In [19]:
#!/usr/bin/env python
# coding: utf-8

import os
import torch
import torch.nn as nn
import torchvision.models as models  # <-- Make sure to add this import for models
from torch.utils.data import random_split
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm






In [29]:
# =========================
# Configuration and Settings
# =========================

class Config:
    # Device configuration
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Data paths
    data_paths = {
        '1_12': r'/content/FTD/Final Testing Dataset',
        '1_4': r'/content/1-4/1-4',
        '5_8': r'/content/5-8/5-8',
        '9_12': r'/content/9-12/9-12',
        'Top': r'/content/Top_level'
    }

    # Training parameters
    training_params = {
        'batch_size': 16,
        'num_workers': 4,
        'epochs': 50
    }

    # Learning rates for different models
    learning_rates = {
        '1_12': 0.001,
        '1_4': 0.0001,
        '5_8': 0.0001,
        '9_12': 0.0001,
        'Top': 0.0001
    }

    # Number of classes for each model
    num_classes = {
        '1_12': 12,
        '1_4': 4,
        '5_8': 4,
        '9_12': 4,
        'Top': 3
    }

    # Model save paths
    model_save_paths = {
        '1_12': '1_12_bats.pth',
        '1_4': '1_4_model.pth',
        '5_8': '5_8_model.pth',
        '9_12': '9_12_model.pth',
        'Top': 'top_model.pth'
    }

# Initialize configuration
cfg = Config()



In [21]:
# =========================
# Data Handling Functions
# =========================

def get_data_transforms(train=True):
    """Define and return data transformations."""
    if train:
        return transforms.Compose([
            transforms.Resize((224, 224)),  # Resize to standard size
            transforms.RandomHorizontalFlip(),  # Random horizontal flipping
            transforms.RandomRotation(45),  # Random rotation
            transforms.ToTensor(),  # Convert to tensor
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
        ])
    else:
        return transforms.Compose([
            transforms.Resize((224, 224)),  # Resize to standard size
            transforms.ToTensor(),  # Convert to tensor
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
        ])

def load_dataset(root_dir, transform):
    """
    Load dataset using ImageFolder.

    Args:
        root_dir (str): Path to the dataset directory.
        transform (torchvision.transforms.Compose): Transformations to apply.

    Returns:
        torch.utils.data.Dataset: Loaded dataset.
    """
    return datasets.ImageFolder(root=root_dir, transform=transform)

def split_dataset(dataset, val_ratio=0.2, test_ratio=0.2):
    """
    Split dataset into training, validation, and test sets.

    Args:
        dataset (torch.utils.data.Dataset): The dataset to split.
        val_ratio (float): Fraction of data for validation.
        test_ratio (float): Fraction of data for testing.

    Returns:
        tuple: train_dataset, val_dataset, test_dataset
    """
    total_size = len(dataset)
    val_size = int(val_ratio * total_size)
    test_size = int(test_ratio * total_size)
    train_size = total_size - val_size - test_size
    train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

    return train_dataset, val_dataset, test_dataset

def get_dataloaders(train_dataset, val_dataset, test_dataset, batch_size, num_workers):
    """
    Create DataLoaders for training, validation, and testing.

    Args:
        train_dataset (torch.utils.data.Dataset): Training dataset.
        val_dataset (torch.utils.data.Dataset): Validation dataset.
        test_dataset (torch.utils.data.Dataset): Test dataset.
        batch_size (int): Batch size.
        num_workers (int): Number of subprocesses for data loading.

    Returns:
        tuple: train_loader, val_loader, test_loader
    """
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)

    return train_loader, val_loader, test_loader


In [22]:
# =========================
# Training and Evaluation Functions
# =========================

def train_model(model, train_loader, val_loader, loss_fn, optimizer, device, epochs, model_name):
    """
    Train the given model and validate after each epoch.

    Args:
        model (nn.Module): The model to train.
        train_loader (DataLoader): DataLoader for training data.
        val_loader (DataLoader): DataLoader for validation data.
        loss_fn (nn.Module): Loss function.
        optimizer (torch.optim.Optimizer): Optimizer.
        device (torch.device): Device to train on.
        epochs (int): Number of epochs.
        model_name (str): Name identifier for the model.

    Returns:
        nn.Module: Trained model.
    """
    model.to(device)
    for epoch in range(epochs):
        model.train()
        print(f"Epoch: {epoch + 1}/{epochs} - Model: {model_name}")
        pbar = tqdm(train_loader, desc=f"Training Epoch {epoch+1}")
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in pbar:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            pbar.set_postfix({'Loss': f"{running_loss / (total):.4f}",
                              'Accuracy': f"{correct / total:.4f}"})

        # Validation after each epoch
        val_acc, val_loss = evaluate_model(model, val_loader, loss_fn, device)
        print(f"Validation - Accuracy: {val_acc:.4f}, Loss: {val_loss:.4f}\n")

    return model

def evaluate_model(model, data_loader, loss_fn, device):
    """
    Evaluate the model on the given dataset.

    Args:
        model (nn.Module): The model to evaluate.
        data_loader (DataLoader): DataLoader for the dataset.
        loss_fn (nn.Module): Loss function.
        device (torch.device): Device to evaluate on.

    Returns:
        tuple: Accuracy, Average Loss
    """
    model.eval()
    correct = 0
    total = 0
    total_loss = 0.0
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    avg_loss = total_loss / len(data_loader)
    return accuracy, avg_loss



In [23]:
# =========================
# Combined Model Class
# =========================

class CombinedModel(nn.Module):
    """
    Combined model that uses a top-level model to decide which specialized model to use for each input.
    """
    def __init__(self, top_model, specialized_models, device):
        """
        Initialize the CombinedModel.

        Args:
            top_model (nn.Module): The top-level model.
            specialized_models (dict): Dictionary of specialized models.
            device (torch.device): Device to run the models on.
        """
        super(CombinedModel, self).__init__()
        self.top_model = top_model
        self.specialized_models = nn.ModuleDict(specialized_models)
        self.device = device

    def forward(self, x):
        """
        Forward pass through the combined model.

        Args:
            x (torch.Tensor): Input tensor.

        Returns:
            torch.Tensor: Combined output tensor.
        """
        # Get decisions from the top-level model
        decision_logits = self.top_model(x)
        decisions = torch.argmax(decision_logits, dim=1)  # Shape: (batch_size,)

        batch_size = x.size(0)
        total_classes = sum([cfg.num_classes[key] for key in self.specialized_models.keys()])
        combined_outputs = torch.zeros(batch_size, total_classes).to(self.device)

        # Process inputs in batches based on decisions
        for decision, model_key in enumerate(self.specialized_models.keys()):
            indices = (decisions == decision).nonzero(as_tuple=True)[0]
            if len(indices) == 0:
                continue  # No samples for this decision
            subset = x[indices]
            outputs = self.specialized_models[model_key](subset)
            # Determine the class index offset
            class_offset = sum([cfg.num_classes[key] for key in list(self.specialized_models.keys())[:decision]])
            combined_outputs[indices, class_offset:class_offset + cfg.num_classes[model_key]] = outputs

        return combined_outputs


In [24]:
# =========================
# Train and Save Model Function
# =========================


def train_and_save_model(model_key, model, train_loader, val_loader):
    """
    Train and save a model given its key, train_loader, and val_loader.

    Args:
        model_key (str): The identifier for the model (e.g., '1_4', '5_8', '9_12').
        model (nn.Module): The model to train.
        train_loader (DataLoader): DataLoader for training.
        val_loader (DataLoader): DataLoader for validation.
    """
    print(f"Training model {model_key}...")

    # Define loss function and optimizer
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=cfg.learning_rates[model_key])

    # Check if model file exists
    model_path = cfg.model_save_paths[model_key]

    if os.path.exists(model_path):
        print(f"Loading existing {model_key} model...")
        model.load_state_dict(torch.load(model_path))
    else:
        print(f"Training new {model_key} model...")
        # Train the model
        model = train_model(model, train_loader, val_loader, loss_fn, optimizer, cfg.device, epochs=cfg.training_params['epochs'], model_name=model_key)

        # Save the model after training
        torch.save(model.state_dict(), model_path)
        print(f"{model_key} model saved successfully!")

In [25]:
# =========================
# Early Stopper Class
# =========================

class EarlyStopping:
    def __init__(self, patience=5, delta=0):
        """
        Args:
            patience (int): How many epochs to wait after last time validation loss improved.
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
        """
        self.patience = patience
        self.delta = delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.delta:
            self.counter += 1
            print(f"EarlyStopping counter: {self.counter} out of {self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0


In [26]:
# =========================
# Main Execution Flow
# =========================

def main():
    # Define transformations for image processing
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize images
        transforms.ToTensor(),  # Convert to tensor
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
    ])

    # Check and print the directory structure and file counts
    print("Verifying dataset structure and files:")
    for root, dirs, files in os.walk(cfg.data_paths['1_12']):
        print(root, len(files))  # Shows number of files in each directory

    # Load the test dataset for combined evaluation
    test_root_dir = cfg.data_paths['1_12']
    test_dataset = load_dataset(test_root_dir, transform)
    _, _, combined_test_loader = get_dataloaders(
        *split_dataset(test_dataset),
        cfg.training_params['batch_size'],
        cfg.training_params['num_workers']
    )

    # Define the top-level model (you can choose any architecture or custom model)
    top_model = models.resnet18(pretrained=False)  # Example: ResNet-18
    top_model.fc = nn.Linear(top_model.fc.in_features, cfg.num_classes['Top'])  # Adjust the output for 3 classes

    # Define specialized models (one for each subset of classes)
    specialized_models = {
        '1_4': models.resnet18(pretrained=False),
        '5_8': models.resnet18(pretrained=False),
        '9_12': models.resnet18(pretrained=False)
    }

    # Adjust the output layers of each specialized model
    specialized_models['1_4'].fc = nn.Linear(specialized_models['1_4'].fc.in_features, cfg.num_classes['1_4'])
    specialized_models['5_8'].fc = nn.Linear(specialized_models['5_8'].fc.in_features, cfg.num_classes['5_8'])
    specialized_models['9_12'].fc = nn.Linear(specialized_models['9_12'].fc.in_features, cfg.num_classes['9_12'])

    # Define the CombinedModel
    combined_model = CombinedModel(top_model, specialized_models, cfg.device)
    combined_model.to(cfg.device)

    # Define the loss function and optimizer
    loss_fn = nn.CrossEntropyLoss()  # For classification tasks
    optimizer = torch.optim.Adam(combined_model.parameters(), lr=0.001)

    # Check if model file exists
    model_path = "combined_model.pth"

    if os.path.exists(model_path):
        print("Loading existing model...")
        combined_model.load_state_dict(torch.load(model_path))
    else:
        print("Training new model...")
        # Train the model if not already saved
        combined_model = train_model(combined_model, combined_test_loader, combined_test_loader, loss_fn, optimizer, cfg.device, epochs=10, model_name='combined_model')

        # Save the model after training
        torch.save(combined_model.state_dict(), model_path)
        print("Model saved successfully!")

    # Evaluate the combined model
    evaluate_model(combined_model, combined_test_loader, loss_fn, cfg.device)

    # Save the evaluated model
    torch.save(combined_model.state_dict(), "combined_model_evaluated.pth")

if __name__ == "__main__":
    main()


Verifying dataset structure and files:
/content/FTD/Final Testing Dataset 0
/content/FTD/Final Testing Dataset/12 500
/content/FTD/Final Testing Dataset/9 500
/content/FTD/Final Testing Dataset/1 501
/content/FTD/Final Testing Dataset/8 500
/content/FTD/Final Testing Dataset/5 500
/content/FTD/Final Testing Dataset/4 500
/content/FTD/Final Testing Dataset/3 500
/content/FTD/Final Testing Dataset/2 500
/content/FTD/Final Testing Dataset/10 500
/content/FTD/Final Testing Dataset/11 500
/content/FTD/Final Testing Dataset/6 500
/content/FTD/Final Testing Dataset/7 500
Training new model...
Epoch: 1/10 - Model: combined_model


Training Epoch 1: 100%|██████████| 150/150 [00:08<00:00, 18.36it/s, Loss=0.2767, Accuracy=0.2267]


Validation - Accuracy: 0.2433, Loss: 2.0876

Epoch: 2/10 - Model: combined_model


Training Epoch 2: 100%|██████████| 150/150 [00:07<00:00, 19.09it/s, Loss=0.2537, Accuracy=0.2625]


Validation - Accuracy: 0.2650, Loss: 2.2043

Epoch: 3/10 - Model: combined_model


...

Validation - Accuracy: 0.3225, Loss: 1.8269

Epoch: 9/10 - Model: combined_model


Training Epoch 9: 100%|██████████| 150/150 [00:08<00:00, 17.60it/s, Loss=0.2232, Accuracy=0.3292]


Validation - Accuracy: 0.3400, Loss: 1.8376

Epoch: 10/10 - Model: combined_model


Training Epoch 10: 100%|██████████| 150/150 [00:08<00:00, 17.67it/s, Loss=0.2185, Accuracy=0.3408]


Validation - Accuracy: 0.3542, Loss: 1.6727

Model saved successfully!


In [30]:
# =========================
# Main Execution Flow (Updated)
# =========================

def main():
    # Define transformations for image processing with augmentation for training
    train_transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize images
        transforms.RandomHorizontalFlip(),  # Random horizontal flipping
        transforms.RandomVerticalFlip(),  # Random vertical flipping
        transforms.RandomRotation(45),  # Random rotation
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Color jittering
        transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),  # Random crop with resizing
        transforms.ToTensor(),  # Convert to tensor
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
    ])

    # Transformation without augmentation for validation and testing
    test_transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize images
        transforms.ToTensor(),  # Convert to tensor
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
    ])

    # Models and their corresponding datasets
    model_info = {
        '1_4': cfg.data_paths['1_4'],
        '5_8': cfg.data_paths['5_8'],
        '9_12': cfg.data_paths['9_12'],
        '1_12': cfg.data_paths['1_12']
    }

    # Train each model separately
    for model_key, dataset_path in model_info.items():
        print(f"\nTraining model {model_key}...")

        # Load the dataset for this model with training and test/val transforms
        dataset = load_dataset(dataset_path, train_transform)
        train_loader, val_loader, test_loader = get_dataloaders(
            *split_dataset(dataset),
            cfg.training_params['batch_size'],
            cfg.training_params['num_workers']
        )

        # Define the model with the appropriate number of classes
        model = models.resnet18(pretrained=False)
        model.fc = nn.Sequential(
            nn.Dropout(0.5),  # Add dropout to reduce overfitting
            nn.Linear(model.fc.in_features, cfg.num_classes[model_key])  # Adjust output layer
        )

        # Define loss function, optimizer, and scheduler
        loss_fn = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=cfg.learning_rates[model_key])
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
        early_stopper = EarlyStopping(patience=5, delta=0.01)

        # Training loop
        for epoch in range(cfg.training_params['epochs']):
            print(f"Epoch: {epoch + 1}/{cfg.training_params['epochs']} - Model: {model_key}")

            # Train for one epoch
            model = train_model(model, train_loader, val_loader, loss_fn, optimizer, cfg.device, epochs=1, model_name=model_key)

            # Validation after each epoch
            val_acc, val_loss = evaluate_model(model, val_loader, loss_fn, cfg.device)
            print(f"Validation - Accuracy: {val_acc:.4f}, Loss: {val_loss:.4f}")

            # Adjust learning rate
            scheduler.step()

            # Early stopping check
            early_stopper(val_loss)
            if early_stopper.early_stop:
                print("Early stopping triggered.")
                break

        # Save the trained model
        torch.save(model.state_dict(), cfg.model_save_paths[model_key])
        print(f"{model_key} model saved successfully!")

        # Evaluate on the test set
        print(f"Evaluating {model_key} model on test data...")
        test_acc, test_loss = evaluate_model(model, test_loader, loss_fn, cfg.device)
        print(f"Test - Accuracy: {test_acc:.4f}, Loss: {test_loss:.4f}")

if __name__ == "__main__":
    main()



Training model 1_4...
Epoch: 1/50 - Model: 1_4
Epoch: 1/1 - Model: 1_4


Training Epoch 1: 100%|██████████| 76/76 [00:09<00:00,  7.82it/s, Loss=0.0608, Accuracy=0.5146]


Validation - Accuracy: 0.6775, Loss: 0.7219

Validation - Accuracy: 0.6325, Loss: 0.7625
Epoch: 2/50 - Model: 1_4
Epoch: 1/1 - Model: 1_4


Training Epoch 1: 100%|██████████| 76/76 [00:09<00:00,  8.05it/s, Loss=0.0458, Accuracy=0.6661]


Validation - Accuracy: 0.6750, Loss: 0.7857

Validation - Accuracy: 0.6525, Loss: 0.8375
EarlyStopping counter: 1 out of 5
Epoch: 3/50 - Model: 1_4
Epoch: 1/1 - Model: 1_4


...

Validation - Accuracy: 0.7733, Loss: 0.5681

Validation - Accuracy: 0.7942, Loss: 0.5535
EarlyStopping counter: 3 out of 5
Epoch: 29/50 - Model: 1_12
Epoch: 1/1 - Model: 1_12


Training Epoch 1: 100%|██████████| 226/226 [00:25<00:00,  8.79it/s, Loss=0.0413, Accuracy=0.7312]


Validation - Accuracy: 0.7758, Loss: 0.5651

Validation - Accuracy: 0.7758, Loss: 0.5648
EarlyStopping counter: 4 out of 5
Epoch: 30/50 - Model: 1_12
Epoch: 1/1 - Model: 1_12


Training Epoch 1: 100%|██████████| 226/226 [00:24<00:00,  9.29it/s, Loss=0.0400, Accuracy=0.7370]


Validation - Accuracy: 0.7908, Loss: 0.5559

Validation - Accuracy: 0.7767, Loss: 0.5688
EarlyStopping counter: 5 out of 5
Early stopping triggered.
1_12 model saved successfully!
Evaluating 1_12 model on test data...
Test - Accuracy: 0.7733, Loss: 0.6009
